In [3]:
#https://huggingface.co/docs/datasets/v1.11.0/add_dataset.html

import pandas as pd
import re

# prepare dataset
df=pd.read_csv('../rajoy.csv')
sentences=df["Text"].to_list()
# splits sentences using marks
split_stc=[re.split('(\.\s|\?|\!|;)',str(s)) for s in sentences]
stcs=[]
# joins with the mark again
for l in split_stc:
    liter=iter(l)
    stcs.append([h + next(liter,'') for h in liter])

# unflatten
sentences=[n.strip() for s in stcs for n in s if len(n.strip())>1]

In [4]:
from datasets import load_dataset

df=pd.DataFrame(sentences,columns=["text"])
df.to_json('../rajoy.json', orient="records", lines="true")
rajoy_ds=load_dataset('json',data_files='../rajoy.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/azureuser/.cache/huggingface/datasets/json/default-ab1bdadc4b0b248c/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-ab1bdadc4b0b248c
